I am going to build a Large Language model and yeah ..that's it

#1.EDA


In [ ]:

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.9.0+cu126
tiktoken version: 0.12.0


In [ ]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [ ]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

class TextdatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
        return len(self.input_ids)

  def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
 def __len__(self):
        return len(self.input_ids)

 def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


In [ ]:
def create_dataloader_v1(txt, batch_size, max_length, stride,
                         shuffle=True, drop_last=True, num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")

    dataset = TextdatasetV1atasetV1(txt, tokenizer, max_length, stride)


    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader


In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

vocab_size = 50257
output_dim = 256
context_length = 1024


token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

def create_dataloader_v1(txt, batch_size, max_length, stride,
                         shuffle=True, drop_last=True, num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")

    # Corrected typo here
    dataset = TextdatasetV1(txt, tokenizer, max_length, stride)


    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

batch_size = 8
max_length = 4
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=batch_size,
    max_length=max_length,
    stride=max_length
)

In [ ]:
 for batch in dataloader:
  x,y = batch

  token_embeddingd=token_embedding_layer(x)
  pos_embeddings=pos_embedding_layer(torch.arange(max_length))

  input_embeddings=token_embeddingd+pos_embeddings

  break

In [ ]:
print(input_embeddings.shape)

torch.Size([8, 4, 256])


#1.Attention Mechanisms

In [1]:
from importlib.metadata import version

In [2]:
import torch
inputs=torch.tensor( [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [4]:
query=inputs[1]
attn_scores_2=torch.empty(inputs.shape[0])
for i , x_1 in enumerate(inputs):
  attn_scores_2[i]=torch.dot(x_1 , query)

  print(attn_scores_2)

tensor([9.5440e-01, 4.3536e-41, 2.8320e-24, 0.0000e+00, 0.0000e+00, 0.0000e+00])
tensor([9.5440e-01, 1.4950e+00, 2.8320e-24, 0.0000e+00, 0.0000e+00, 0.0000e+00])
tensor([0.9544, 1.4950, 1.4754, 0.0000, 0.0000, 0.0000])
tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.0000, 0.0000])
tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 0.0000])
tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [5]:
res =0

for idx , element in enumerate (inputs[0]):
    res+=inputs[0][idx ]*query[idx]

    print(res)
    print(torch.dot(inputs[0],query))

tensor(0.2365)
tensor(0.9544)
tensor(0.3670)
tensor(0.9544)
tensor(0.9544)
tensor(0.9544)


In [6]:
attn_weights_2_tmp=attn_scores_2/attn_scores_2.sum()
print("Attetion weights:", attn_weights_2_tmp)
print("sum:",attn_weights_2_tmp.sum())

Attetion weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
sum: tensor(1.0000)


In [8]:
def softmax_naive(x):
  return torch.exp(x)/torch.exp(x).sum(dim=0)

  attn_weights_2_naive=softmax_naive(attn_scores_2)
  print("attention weights:",attn_weights_2_naive)
  print("sum:", attn_weights_2_naive.sum())

In [9]:
attn_weights_2=torch.softmax(attn_scores_2,dim=0)

print("Attention weights:",attn_weights_2)
print("sum:",attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
sum: tensor(1.)


In [10]:
#compute theb context vector
query=inputs[1]
context_vec_2=torch.zeros(query.shape)
for i ,x_i in enumerate(inputs):
  context_vec_2+=attn_weights_2[i]*x_i

  print(context_vec_2)

tensor([0.0596, 0.0208, 0.1233])
tensor([0.1904, 0.2277, 0.2803])
tensor([0.3234, 0.4260, 0.4296])
tensor([0.3507, 0.4979, 0.4705])
tensor([0.4340, 0.5250, 0.4813])
tensor([0.4419, 0.6515, 0.5683])


In [12]:
attn_scores=torch.empty(6,6)
for i , x_i in enumerate (inputs):
  for j,x_j in enumerate(inputs):
    attn_scores[i,j]=torch.dot(x_i,x_i)
print(attn_scores)

tensor([[0.9995, 0.9995, 0.9995, 0.9995, 0.9995, 0.9995],
        [1.4950, 1.4950, 1.4950, 1.4950, 1.4950, 1.4950],
        [1.4570, 1.4570, 1.4570, 1.4570, 1.4570, 1.4570],
        [0.4937, 0.4937, 0.4937, 0.4937, 0.4937, 0.4937],
        [0.6654, 0.6654, 0.6654, 0.6654, 0.6654, 0.6654],
        [0.9450, 0.9450, 0.9450, 0.9450, 0.9450, 0.9450]])


In [14]:
attn_scores=inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [16]:
attn_weights=torch.softmax(attn_scores,dim=1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [17]:
all_context_vectors=attn_weights @ inputs
print(all_context_vectors)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [21]:
x_2=inputs[1]
d_in=inputs.shape[1]
d_out=2

torch.manual_seed(123)
w_query = torch.randn(d_in, d_out)
w_key = torch.randn(d_in, d_out)
w_value = torch.randn(d_in, d_out)

query = inputs @ w_query
keys = inputs @ w_key
values = inputs @ w_value

attn_scores_2=query[1] @ keys.T
print(attn_scores_2)

d_k=keys.shape[1]
attn_weights_2=torch.softmax(attn_scores_2/d_k**0.5,dim=-1)
print(attn_weights)

tensor([ 0.2172,  0.1376,  0.1730, -0.0491,  0.7616, -0.3809])
tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])
